In [4]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer

#NLTK-------------------------------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split


# Import libraries for feature 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.feature_selection import SequentialFeatureSelector as SFS


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")


from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [8]:
#Read files
textfile = r'/gdrive/My Drive/CIS508/Assignment 5/Comments.csv'
textData = pd.read_csv(textfile) #creates a dataframe

CustInfofile = r'/gdrive/My Drive/CIS508/Assignment 5/Customers.csv'
CustInfoData = pd.read_csv(CustInfofile)  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)


(2070, 2)
(2070, 17)


In [9]:
#Extract target column from Customer Info file
y_train = CustInfoData["TARGET"]
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(textData.shape)
textData.head()
print(y_train)

(2070, 16)
(2070, 2)
0       Cancelled
1         Current
2         Current
3         Current
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


In [0]:
##----------------SNOWBALL STEMMER----------------
# Use English stemmer
stemmer = SnowballStemmer("english")
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize) #Tokenize is breaking each comment into single word

export_csv = textData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TextDataTokenized1.csv')

#Now do stemming - create a new dataframe to store stemmed version
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/newTextDataTS.csv')

#Join stemmed strings
newTextData['CommentsTokenizedStemmed'] = newTextData['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/newTextData-Joined.csv')



In [13]:
#Do Bag-Of-Words model - Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
#count_vect = CountVectorizer(stop_words=None)
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts = count_vect.fit_transform(newTextData.CommentsTokenizedStemmed)
print(TD_counts.shape)
TD_counts.dtype
print(count_vect.get_feature_names())
#print(TD_counts)
DF_TD_Counts=pd.DataFrame(TD_counts.toarray())
#print(DF_TD_Counts)
#export_csv = DF_TD_Counts.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TD_counts-TokenizedStemmed.csv')


(2070, 354)
['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effect', '

In [0]:
##----------------PORTER STEMMER----------------
stemmer = PorterStemmer()
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize) #Tokenize is breaking each comment into single word

#export_csv = textData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TextDataTokenized1.csv')

#Now do stemming - create a new dataframe to store stemmed version
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/newTextDataTS_Porter.csv')

#Join stemmed strings
newTextData['CommentsTokenizedStemmed'] = newTextData['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/newTextData-JoinedPorter.csv')



In [15]:
#Do Bag-Of-Words model - Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
#count_vect = CountVectorizer(stop_words=None)
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts_Porter = count_vect.fit_transform(newTextData.CommentsTokenizedStemmed)
print(TD_counts_Porter.shape)
TD_counts_Porter.dtype
print(count_vect.get_feature_names())
#print(TD_counts)
DF_TD_Counts=pd.DataFrame(TD_counts.toarray())
#print(DF_TD_Counts)
#export_csv = DF_TD_Counts.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TD_counts-TokenizedStemmed_Porter.csv')

(2070, 366)
['3399', '3g', 'As', 'CC', 'He', 'If', 'In', 'Is', 'It', 'We', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constanli', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'd

In [0]:
##----------------LANCASTER STEMMER----------------
stemmer = LancasterStemmer()
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize) #Tokenize is breaking each comment into single word

#export_csv = textData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TextDataTokenized1.csv')

#Now do stemming - create a new dataframe to store stemmed version
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/newTextDataTS_Lancaster.csv')

#Join stemmed strings
newTextData['CommentsTokenizedStemmed'] = newTextData['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/newTextData-JoinedLancaster.csv')

In [20]:
#Do Bag-Of-Words model - Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts_Lancaster = count_vect.fit_transform(newTextData.CommentsTokenizedStemmed)
print(TD_counts_Lancaster.shape)
TD_counts_Lancaster.dtype
print(count_vect.get_feature_names())
#print(TD_counts)
DF_TD_Counts=pd.DataFrame(TD_counts.toarray())
print(DF_TD_Counts)
#export_csv = DF_TD_Counts.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TD_counts-TokenizedStemmed_Lancaster.csv')

(2070, 354)
['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effect', '

In [18]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts)
print(X_train_tfidf.shape)
DF_TF_IDF=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF)
export_csv= DF_TF_IDF.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TFIDF_counts-TokenizedStemmed.csv')


(2070, 354)
      0    1    2    3        4    5    ...  348  349  350  351  352  353
0     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.27568  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...  ...      ...  ...  ...  ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0

[2070 rows x 354 columns]


In [37]:
#Feature selection using Filter method
new_DF_TF_IDF = SelectKBest(score_func=chi2, k=10)
new_DF_TF_IDF.fit_transform(DF_TF_IDF,y_train)

cols = new_DF_TF_IDF.get_support(indices=True)
DF_TF_IDF_SelectedFeatures = DF_TF_IDF[cols]
print(DF_TF_IDF_SelectedFeatures)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)

export_csv= DF_TF_IDF_SelectedFeatures.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/TFIDF_counts-Selected Features_Filter.csv')


      14        50   70        115       121  212  239  259  313  319
0     0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
1     0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
2     0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
3     0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
4     0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
...   ...       ...  ...       ...       ...  ...  ...  ...  ...  ...
2065  0.0  0.446161  0.0  0.460113  0.457852  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0

[2070 rows x 10 columns]


In [38]:
#Merge files

print(CustInfoData.shape)
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
print(X_train.shape)
combined=pd.concat([X_train, DF_TF_IDF_SelectedFeatures], axis=1)
print(combined.shape)
print(combined)
export_csv= combined.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/Combined-Cust+TFIDF+SelectedFeatures.csv')

(2070, 17)
(2070, 16)
(2070, 26)
        ID Sex Status  Children  Est_Income  ...  212  239  259  313  319
0        1   F      S         1    38000.00  ...  0.0  0.0  0.0  0.0  0.0
1        6   M      M         2    29616.00  ...  0.0  0.0  0.0  0.0  0.0
2        8   M      M         0    19732.80  ...  0.0  0.0  0.0  0.0  0.0
3       11   M      S         2       96.33  ...  0.0  0.0  0.0  0.0  0.0
4       14   F      M         2    52004.80  ...  0.0  0.0  0.0  0.0  0.0
...    ...  ..    ...       ...         ...  ...  ...  ...  ...  ...  ...
2065  3821   F      S         0    78851.30  ...  0.0  0.0  0.0  0.0  0.0
2066  3822   F      S         1    17540.70  ...  0.0  0.0  0.0  0.0  0.0
2067  3823   F      M         0    83891.90  ...  0.0  0.0  0.0  0.0  0.0
2068  3824   F      M         2    28220.80  ...  0.0  0.0  0.0  0.0  0.0
2069  3825   F      S         0    28589.10  ...  0.0  0.0  0.0  0.0  0.0

[2070 rows x 26 columns]


In [39]:
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
#X_cat = combined.select_dtypes(exclude=['int','float64'])
print(X_cat)
combined_one_hot = pd.get_dummies(combined,columns=X_cat)
print(combined_one_hot.shape)
export_csv= combined_one_hot.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/combined_one_hot.csv')

['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 34)


In [40]:
#Test/Train split
df = pd.DataFrame(combined_one_hot)
Y = y_train

X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(df, Y, test_size = 0.2)

print(X_train_i.shape)
print(X_test_i.shape)

print(y_train_i.shape)
print(y_test_i.shape)

(1656, 34)
(414, 34)
(1656,)
(414,)


In [42]:
#Construct a Random Forest Classifier on combined data
clf=RandomForestClassifier()
RF_Comb = clf.fit(X_train_i,y_train_i)
print("Accuracy score (testing): {0:.6f}".format(clf.score(X_test_i, y_test_i)))
rf_predictions = clf.predict(X_test_i)
print("Confusion Matrix:")
print(confusion_matrix(y_test_i, rf_predictions))
print("Classification Report")
print(classification_report(y_test_i, rf_predictions))

Accuracy score (testing): 0.879227
Confusion Matrix:
[[147  17]
 [ 33 217]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.82      0.90      0.85       164
     Current       0.93      0.87      0.90       250

    accuracy                           0.88       414
   macro avg       0.87      0.88      0.88       414
weighted avg       0.88      0.88      0.88       414



In [43]:
#Feature selection using Wrapper method
clf = RandomForestClassifier()
sfs1 = SFS(clf, 
           k_features=20, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=3)

sfs1 = sfs1.fit(DF_TF_IDF,y_train)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 354 out of 354 | elapsed:   22.3s finished

[2019-12-14 02:29:44] Features: 1/20 -- score: 0.6154589371980678[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 353 out of 353 | elapsed:   22.5s finished

[2019-12-14 02:30:07] Features: 2/20 -- score: 0.6198067632850242[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 352 out of 352 | elapsed:   22.7s finished

[2019-12-14 02:30:30] Features: 3/20 -- score: 0.6222222222222222[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [44]:
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[4, 10, 11, 14, 20, 44, 48, 71, 109, 118, 130, 143, 144, 155, 239, 289, 313, 319, 338, 342]


In [0]:
#Merge files

print(CustInfoData.shape)
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
print(X_train.shape)
combined=pd.concat([X_train, DF_TF_IDF[feat_cols]], axis=1)
print(combined.shape)
print(combined)
#export_csv= combined.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/Combined-Cust+TFIDF+SelectedFeatures.csv')



(2070, 17)
(2070, 16)
(2070, 36)
        ID Sex Status  Children  Est_Income  ...  278       294  295  299  351
0        1   F      S         1    38000.00  ...  0.0  0.000000  0.0  0.0  0.0
1        6   M      M         2    29616.00  ...  0.0  0.000000  0.0  0.0  0.0
2        8   M      M         0    19732.80  ...  0.0  0.000000  0.0  0.0  0.0
3       11   M      S         2       96.33  ...  0.0  0.000000  0.0  0.0  0.0
4       14   F      M         2    52004.80  ...  0.0  0.000000  0.0  0.0  0.0
...    ...  ..    ...       ...         ...  ...  ...       ...  ...  ...  ...
2065  3821   F      S         0    78851.30  ...  0.0  0.000000  0.0  0.0  0.0
2066  3822   F      S         1    17540.70  ...  0.0  0.372781  0.0  0.0  0.0
2067  3823   F      M         0    83891.90  ...  0.0  0.000000  0.0  0.0  0.0
2068  3824   F      M         2    28220.80  ...  0.0  0.000000  0.0  0.0  0.0
2069  3825   F      S         0    28589.10  ...  0.0  0.372781  0.0  0.0  0.0

[2070 rows x 36 co

In [0]:
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
#X_cat = combined.select_dtypes(exclude=['int','float64'])
print(X_cat)
combined_one_hot = pd.get_dummies(combined,columns=X_cat)
print(combined_one_hot.shape)
#export_csv= combined_one_hot.to_csv(r'/gdrive/My Drive/CIS508/Assignment 5/combined_one_hot.csv')



['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 44)


In [0]:
#Test/Train split
df = pd.DataFrame(combined_one_hot)
Y = y_train

X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(df, Y, test_size = 0.2)

print(X_train_i.shape)
print(X_test_i.shape)

print(y_train_i.shape)
print(y_test_i.shape)

(1656, 44)
(414, 44)
(1656,)
(414,)


In [0]:
#Construct a Random Forest Classifier on combined data
clf=RandomForestClassifier()
RF_Comb = clf.fit(X_train_i,y_train_i)
print("Accuracy score (testing): {0:.6f}".format(clf.score(X_test_i, y_test_i)))
rf_predictions = clf.predict(X_test_i)
print("Confusion Matrix:")
print(confusion_matrix(y_test_i, rf_predictions))
print("Classification Report")
print(classification_report(y_test_i, rf_predictions))

Accuracy score (testing): 0.891304
Confusion Matrix:
[[137  19]
 [ 26 232]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.84      0.88      0.86       156
     Current       0.92      0.90      0.91       258

    accuracy                           0.89       414
   macro avg       0.88      0.89      0.89       414
weighted avg       0.89      0.89      0.89       414

